# Quais deputados se autobeneficiaram ao votar a favor da lei de improbidade administrativa
O código vai buscar informações, via API da Câmara dos Deputados, da votação do PL n° 2505, de 2021 - sob ID DE VOTAÇÃO: 2184458-184. <br>
Na 2ª etapa, iremos automatizar searchs no e-SAJ do TJ-MG e raspar as informações. 

## Preparando o ambiente
Para a raspagem dos dados, utilizaremos as bibliotecas Selenium. Para a análise, utilizaremos o Pandas. 
Já para a representação dos dados utilizaremos o matplotlib e Seaborn. 

In [1]:

#Biblioteca para raspagem 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

#Bibliotecas para várias funções relacionadas à tempo
import time
from datetime import datetime

#Bibliotecas para análise dos dados
import pandas as pd 
import numpy as np 
import requests as requests
import matplotlib.pyplot as plt 
import seaborn as sns


In [1]:
#Linhas para configurar a IDE para exibir o máximo de colunas e largura máxima de colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

NameError: name 'pd' is not defined

# Conexão API Câmara dos Deputados

In [3]:
#Vamos criar nosso cabeçalho para aceitar as requisiões em json
headers = {
    "Accept": "Application/json"
}

### Identificar os id's da PL

In [4]:
votos_pl = requests.get('https://dadosabertos.camara.leg.br/api/v2/proposicoes/2184458/votacoes?ordem=DESC&ordenarPor=dataHoraRegistro', headers=headers)
votos_pl = votos_pl.json()

votos_pl = pd.json_normalize(votos_pl['dados'])
votos_pl.head(6)


,id,uri,data,dataHoraRegistro,siglaOrgao,uriOrgao,uriEvento,proposicaoObjeto,uriProposicaoObjeto,descricao,aprovacao
0,2184458-186,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-186,2021-10-06,2021-10-06T16:58:25,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63505,None,None,"Aprovada a Redação Final assinada pelo Relator, Dep. Carlos Zarattini (PT-SP).",1.0
1,2184458-184,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-184,2021-10-06,2021-10-06T16:57:25,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63505,None,None,Aprovada a Emenda do Senado Federal nº 8. Sim: 287; não: 133; abstenção: 2; total: 422.,1.0
2,2184458-172,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-172,2021-10-05,2021-10-05T20:05:40,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63472,None,None,"Rejeitada a Emenda do Senado Federal nº 4 ao Projeto de Lei nº 2.505, de 2021, ressalvados os destaques. Sim: 162; não: 253; abstenção: 4; total: 419.",0.0
3,2184458-169,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-169,2021-10-05,2021-10-05T19:47:45,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63472,None,None,"Aprovadas as Emendas do Senado Federal nºs 1, 2, 3, 5, 6, 7 e 8 ao Projeto de Lei nº 2.505, de 2021, ressalvados os destaques. Sim: 395; não: 22; abstenção: 3; total: 420.",1.0
4,2184458-161,https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-161,2021-10-05,2021-10-05T18:34:35,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/eventos/63472,None,None,Rejeitado o Requerimento. Sim: 60; não: 346; total: 406.,0.0
5,2301348-3,https://dadosabertos.camara.leg.br/api/v2/votacoes/2301348-3,2021-10-05,2021-10-05T13:23:52,CPD,https://dadosabertos.camara.leg.br/api/v2/orgaos/537480,https://dadosabertos.camara.leg.br/api/v2/eventos/63437,REQ 76/2021 CPD,https://dadosabertos.camara.leg.br/api/v2/proposicoes/2301348,Aprovado.,1.0


### Lista votos

In [5]:
lista_votos = requests.get('https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-184/votos', headers=headers)
lista_votos = lista_votos.json()

lista_votos = pd.json_normalize(lista_votos['dados'])
lista_votos

,tipoVoto,dataRegistroVoto,deputado_.id,deputado_.uri,deputado_.nome,deputado_.siglaPartido,deputado_.uriPartido,deputado_.siglaUf,deputado_.idLegislatura,deputado_.urlFoto,deputado_.email
0,Sim,2021-10-06T16:55:06,74352,https://dadosabertos.camara.leg.br/api/v2/deputados/74352,Nilson Pinto,PSDB,https://dadosabertos.camara.leg.br/api/v2/partidos/36835,PA,56,https://www.camara.leg.br/internet/deputado/bandep/74352.jpg,None
1,Sim,2021-10-06T16:54:47,74366,https://dadosabertos.camara.leg.br/api/v2/deputados/74366,João Campos,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/partidos/37908,GO,56,https://www.camara.leg.br/internet/deputado/bandep/74366.jpg,None
2,Sim,2021-10-06T16:54:10,204553,https://dadosabertos.camara.leg.br/api/v2/deputados/204553,Pastor Sargento Isidório,AVANTE,https://dadosabertos.camara.leg.br/api/v2/partidos/36898,BA,56,https://www.camara.leg.br/internet/deputado/bandep/204553.jpg,dep.pastorsargentoisidorio@camara.leg.br
3,Não,2021-10-06T16:54:02,141553,https://dadosabertos.camara.leg.br/api/v2/deputados/141553,Vanderlei Macris,PSDB,https://dadosabertos.camara.leg.br/api/v2/partidos/36835,SP,56,https://www.camara.leg.br/internet/deputado/bandep/141553.jpg,None
4,Não,2021-10-06T16:53:57,204363,https://dadosabertos.camara.leg.br/api/v2/deputados/204363,Osires Damaso,PSC,https://dadosabertos.camara.leg.br/api/v2/partidos/36833,TO,56,https://www.camara.leg.br/internet/deputado/bandep/204363.jpg,None
...,...,...,...,...,...,...,...,...,...,...,...
418,Não,2021-10-06T16:45:03,204548,https://dadosabertos.camara.leg.br/api/v2/deputados/204548,Heitor Freire,PSL,https://dadosabertos.camara.leg.br/api/v2/partidos/36837,CE,56,https://www.camara.leg.br/internet/deputado/bandep/204548.jpg,None
419,Sim,2021-10-06T16:45:01,76874,https://dadosabertos.camara.leg.br/api/v2/deputados/76874,Marcelo Freixo,PSB,https://dadosabertos.camara.leg.br/api/v2/partidos/36832,RJ,56,https://www.camara.leg.br/internet/deputado/bandep/76874.jpg,None
420,Sim,2021-10-06T16:45:01,160556,https://dadosabertos.camara.leg.br/api/v2/deputados/160556,Padre João,PT,https://dadosabertos.camara.leg.br/api/v2/partidos/36844,MG,56,https://www.camara.leg.br/internet/deputado/bandep/160556.jpg,dep.padrejoao@camara.leg.br
421,Sim,2021-10-06T16:44:59,73801,https://dadosabertos.camara.leg.br/api/v2/deputados/73801,Renildo Calheiros,PCdoB,https://dadosabertos.camara.leg.br/api/v2/partidos/36779,PE,56,https://www.camara.leg.br/internet/deputado/bandep/73801.jpg,dep.renildocalheiros@camara.leg.br


### Informações adicionais dos deputados

In [6]:
nome_deputado = []

for i, row in lista_votos.iterrows():
    deputado_id = row['deputado_.id']
    deputados_infos = requests.get(f'https://dadosabertos.camara.leg.br/api/v2/deputados/{deputado_id}')
    deputados_infos = deputados_infos.json()
    nome_deputado.append(deputados_infos['dados']['nomeCivil'])
    print(deputado_id)

lista_votos['NomeCivilDeputado'] = nome_deputado

74352
74366
204553
141553
204363
160758
204505
178934
178884
217480
204547
204414
178896
178927
204364
204387
204531
160592
141516
204473
178996
204482
160591
204395
141513
141458
178986
141434
157130
204559
160976
196358
178937
178956
204527
74090
204398
204428
141459
74317
204377
204515
178933
73692
204539
160610
198783
204392
74376
166402
160553
179001
74537
74253
160527
217330
74439
178910
74646
204571
73486
74052
209189
204449
204431
178964
178921
204488
66179
90201
204386
74159
197438
178970
204534
164359
160601
178985
178912
74158
204511
204360
204408
204475
73466
74419
178873
160528
204502
141405
204467
178843
204518
178871
73788
160510
133439
160508
177282
204532
73604
204427
178909
204522
204446
141391
204410
160604
191945
204479
204405
178835
195866
178948
204369
215042
204413
160666
204481
160642
204366
204537
204485
74749
141523
141408
143084
160665
204407
75431
204497
204538
205548
115746
204566
92776
178939
204418
178832
204393
204498
74467
194260
74254
204372
136811
204

# Rodagem Automatizada

In [ ]:
#Vamos criar um DataFrame contendo apenas os deputados de SP
bd_deputados_mg = lista_votos.query('`deputado_.siglaUf` == "MG" and tipoVoto == "Sim"')
bd_deputados_mg

,tipoVoto,dataRegistroVoto,deputado_.id,deputado_.uri,deputado_.nome,deputado_.siglaPartido,deputado_.uriPartido,deputado_.siglaUf,deputado_.idLegislatura,deputado_.urlFoto,deputado_.email,NomeCivilDeputado
26,Sim,2021-10-06T16:51:53,178986,https://dadosabertos.camara.leg.br/api/v2/deputados/178986,Nilto Tatto,PT,https://dadosabertos.camara.leg.br/api/v2/partidos/36844,SP,56,https://www.camara.leg.br/internet/deputado/bandep/178986.jpg,dep.niltotatto@camara.leg.br,NILTO IGNACIO TATTO
30,Sim,2021-10-06T16:51:37,160976,https://dadosabertos.camara.leg.br/api/v2/deputados/160976,Tiririca,PL,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,SP,56,https://www.camara.leg.br/internet/deputado/bandep/160976.jpg,dep.tiririca@camara.leg.br,FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA
34,Sim,2021-10-06T16:51:12,204527,https://dadosabertos.camara.leg.br/api/v2/deputados/204527,Geninho Zuliani,DEM,https://dadosabertos.camara.leg.br/api/v2/partidos/36769,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204527.jpg,None,EUGENIO JOSÉ ZULIANI
76,Sim,2021-10-06T16:49:06,160601,https://dadosabertos.camara.leg.br/api/v2/deputados/160601,Pr. Marco Feliciano,PL,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,SP,56,https://www.camara.leg.br/internet/deputado/bandep/160601.jpg,dep.pr.marcofeliciano@camara.leg.br,MARCO ANTÔNIO FELICIANO
80,Sim,2021-10-06T16:49:00,204511,https://dadosabertos.camara.leg.br/api/v2/deputados/204511,David Soares,DEM,https://dadosabertos.camara.leg.br/api/v2/partidos/36769,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204511.jpg,dep.davidsoares@camara.leg.br,DAVID BEZERRA RIBEIRO SOARES
100,Sim,2021-10-06T16:48:23,73604,https://dadosabertos.camara.leg.br/api/v2/deputados/73604,Rui Falcão,PT,https://dadosabertos.camara.leg.br/api/v2/partidos/36844,SP,56,https://www.camara.leg.br/internet/deputado/bandep/73604.jpg,dep.ruifalcao@camara.leg.br,RUI GOETHE DA COSTA FALCÃO
121,Sim,2021-10-06T16:47:41,204537,https://dadosabertos.camara.leg.br/api/v2/deputados/204537,Enrico Misasi,PV,https://dadosabertos.camara.leg.br/api/v2/partidos/36851,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204537.jpg,None,ENRICO VAN BLARCUM DE GRAAFF MISASI
122,Sim,2021-10-06T16:47:40,204485,https://dadosabertos.camara.leg.br/api/v2/deputados/204485,Luiz Carlos Motta,PL,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,SP,56,https://www.camara.leg.br/internet/deputado/bandep/204485.jpg,dep.luizcarlosmotta@camara.leg.br,LUIZ CARLOS MOTTA
147,Sim,2021-10-06T16:47:18,217036,https://dadosabertos.camara.leg.br/api/v2/deputados/217036,Henrique do Paraíso,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/partidos/37908,SP,56,https://www.camara.leg.br/internet/deputado/bandep/217036.jpg,None,HENRIQUE STEIN SCIASCIO
163,Sim,2021-10-06T16:47:06,73441,https://dadosabertos.camara.leg.br/api/v2/deputados/73441,Celso Russomanno,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/partidos/37908,SP,56,https://www.camara.leg.br/internet/deputado/bandep/73441.jpg,dep.celsorussomanno@camara.leg.br,CELSO UBIRAJARA RUSSOMANNO


In [ ]:
class PjeScraper:
    def __init__(self):
        self.driver = webdriver.Chrome()
        self.driver.get('https://pje-consulta-publica.tjmg.jus.br/pje/login.seam')
        self.driver.maximize_window()
        self.resultados = []

    def realizar_pesquisa(self, nome_deputado):
        try:
            nome_search = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, 'fPP:dnp:nomeParte'))
            )
            nome_search.clear()
            nome_search.send_keys(nome_deputado, Keys.ENTER)

            # Esperar pelos resultados da pesquisa 
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'btn-block'))
            )
            time.sleep(5)
            return True  # Indica sucesso na pesquisa

        except TimeoutException:
            print(f"Tempo de espera excedido ao pesquisar por {nome_deputado}")
            return False
        except NoSuchElementException:
            print(f"Elemento não encontrado ao pesquisar por {nome_deputado}")
            return False
        except Exception as e:
            print(f"Erro desconhecido ao pesquisar por {nome_deputado}: {e}")
            return False

    def extrair_detalhes_processo(self, nome_deputado, partido_deputado):
        try:
            cod_processos = WebDriverWait(self.driver, 15).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'btn-block'))
            )
            
            time.sleep(5)
            
            for processo in cod_processos:
                numero_processo = processo.text  # Extrair número do processo
                self.resultados.append({
                    'nome_deputado': nome_deputado,
                    'partido_deputado': partido_deputado,
                    'numero_processo': numero_processo
                })
                print(nome_deputado, partido_deputado)

        except TimeoutException:
            print(f"Tempo de espera excedido ao extrair detalhes para {nome_deputado}")
        except NoSuchElementException:
            print(f"Elemento não encontrado ao extrair detalhes para {nome_deputado}")
        except Exception as e:
            print(f"Erro desconhecido ao extrair detalhes para {nome_deputado}: {e}")

    def voltar_para_pesquisa(self):
        try:
            nome_search = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, 'fPP:dnp:nomeParte'))
            )
            nome_search.clear()

        except TimeoutException:
            print("Tempo de espera excedido ao limpar campo de pesquisa")
        except NoSuchElementException:
            print("Elemento não encontrado ao limpar campo de pesquisa")
        except Exception as e:
            print(f"Erro desconhecido ao limpar campo de pesquisa: {e}")

    def executar_raspagem(self, df):
        for _, row in df.iterrows():
            nome_deputado = row['NomeCivilDeputado']
            partido_deputado = row['deputado_.siglaPartido']

            if self.realizar_pesquisa(nome_deputado):
                self.extrair_detalhes_processo(nome_deputado, partido_deputado)
                self.voltar_para_pesquisa()

        self.driver.quit()
        return pd.DataFrame(self.resultados)

# Exemplo de uso
scraper = PjeScraper()
df_resultados = scraper.executar_raspagem(bd_deputados_mg)

Pesquisando por: NILTO IGNACIO TATTO (PT)
Nenhum processo encontrado na tabela para NILTO IGNACIO TATTO
Pesquisando por: FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA (PL)
Nenhum processo encontrado na tabela para FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA
Pesquisando por: EUGENIO JOSÉ ZULIANI (DEM)
Pesquisando por: MARCO ANTÔNIO FELICIANO (PL)
Pesquisando por: DAVID BEZERRA RIBEIRO SOARES (DEM)
Pesquisando por: RUI GOETHE DA COSTA FALCÃO (PT)
Nenhum processo encontrado na tabela para RUI GOETHE DA COSTA FALCÃO
Pesquisando por: ENRICO VAN BLARCUM DE GRAAFF MISASI (PV)
Pesquisando por: LUIZ CARLOS MOTTA (PL)
Pesquisando por: HENRIQUE STEIN SCIASCIO (REPUBLICANOS)
Pesquisando por: CELSO UBIRAJARA RUSSOMANNO (REPUBLICANOS)
Pesquisando por: LUIZ PAULO TEIXEIRA FERREIRA (PT)
Pesquisando por: VINICIUS RAPOZO DE CARVALHO (REPUBLICANOS)
Nenhum processo encontrado na tabela para VINICIUS RAPOZO DE CARVALHO
Pesquisando por: PAULO ROBERTO FREIRE DA COSTA (PL)
Pesquisando por: BENEDITO ROBERTO ALVES 

### Ajustes do DataFrame

In [ ]:
df_resultados['numero_processo'].replace('', np.nan, inplace=True)
df_resultados.dropna(subset='numero_processo', inplace=True)
df_resultados[['Descr', 'numero_processo', "assunto_processo"]] = df_resultados['numero_processo'].str.split(" ", n=2, expand=True)